In [1]:
from transformer_lens import HookedTransformer

d:\Doctorado\Papers\Para EL paper\Workspace\synthetic_gender_datasets\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import sys
import os

src_path = os.path.normpath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from utils.dataset import to_token_ids

In [3]:
model_names = ["gpt2-small", "gpt2-medium", "gpt2-large", "facebook/opt-125m", "EleutherAI/gpt-neo-125M"]
source_path = "../datasets/raw_names.csv"
target_path = "../datasets/names.csv"

In [4]:
df = pd.read_csv(source_path)
df.head()

,name,gender,number
0,James,M,5122407
1,John,M,5096818
2,Robert,M,4803587
3,Michael,M,4326215
4,Mary,F,4118147


In [5]:
df_pivoted = (
    df
    .pivot_table(
        index="name",
        columns="gender",
        values="number",
        aggfunc="sum",
        fill_value=0
    )
    .reset_index()
)

In [6]:
total = df_pivoted["F"].sum() + df_pivoted["M"].sum()


df_pivoted["name_weight"] = (df_pivoted["F"] + df_pivoted["M"]) / total

# Calcular proporciones de género
df_pivoted["F_prop"] = df_pivoted["F"] / (df_pivoted["F"] + df_pivoted["M"])
df_pivoted["M_prop"] = 1 - df_pivoted["F_prop"]

# Calcular ponderaciones globales
df_pivoted["F_weighted"] = df_pivoted["F_prop"] * df_pivoted["name_weight"] 
df_pivoted["M_weighted"] = df_pivoted["M_prop"] * df_pivoted["name_weight"] 

# Normalizar ponderaciones
df_pivoted["F_weighted_norm"] = (df_pivoted["F_weighted"] / df_pivoted["F_weighted"].sum()) * 1000
df_pivoted["M_weighted_norm"] = (df_pivoted["M_weighted"] / df_pivoted["M_weighted"].sum()) * 1000


In [8]:
for model_name in model_names:
    model = HookedTransformer.from_pretrained(
        model_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        refactor_factored_attn_matrices=True
    )

    df_pivoted[model_name] = df_pivoted["name"].apply(lambda x: to_token_ids(model, x))
    df_pivoted[f"{model_name}-size"] = df_pivoted[model_name].apply(lambda x: len(x.split(",")))

Loaded pretrained model gpt2-small into HookedTransformer


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Loaded pretrained model gpt2-medium into HookedTransformer


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Loaded pretrained model gpt2-large into HookedTransformer


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Loaded pretrained model facebook/opt-125m into HookedTransformer


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Loaded pretrained model EleutherAI/gpt-neo-125M into HookedTransformer


In [9]:
df_pivoted

gender,name,F,M,name_weight,F_prop,M_prop,F_weighted,M_weighted,F_weighted_norm,M_weighted_norm,gpt2-small,gpt2-small-size,gpt2-medium,gpt2-medium-size,gpt2-large,gpt2-large-size,facebook/opt-125m,facebook/opt-125m-size,EleutherAI/gpt-neo-125M,EleutherAI/gpt-neo-125M-size
0,Aaban,0,87,2.550345e-07,0.0,1.0,0.000000e+00,2.550345e-07,0.000000,0.000505,"317,45094",2,"317,45094",2,"317,45094",2,"83,26528",2,"317,45094",2
1,Aabha,28,0,8.208008e-08,1.0,0.0,8.208008e-08,0.000000e+00,0.000166,0.000000,"317,397,3099",3,"317,397,3099",3,"317,397,3099",3,"83,873,1999",3,"317,397,3099",3
2,Aabid,0,5,1.465716e-08,0.0,1.0,0.000000e+00,1.465716e-08,0.000000,0.000029,"317,397,312",3,"317,397,312",3,"317,397,312",3,"83,873,808",3,"317,397,312",3
3,Aabriella,15,0,4.397147e-08,1.0,0.0,4.397147e-08,0.000000e+00,0.000089,0.000000,"317,397,380,12627",4,"317,397,380,12627",4,"317,397,380,12627",4,"83,873,1069,8461",4,"317,397,380,12627",4
4,Aada,5,0,1.465716e-08,1.0,0.0,1.465716e-08,0.000000e+00,0.000030,0.000000,"317,4763",2,"317,4763",2,"317,4763",2,"83,2095",2,"317,4763",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95021,Zyvion,0,5,1.465716e-08,0.0,1.0,0.000000e+00,1.465716e-08,0.000000,0.000029,"40905,85,295",3,"40905,85,295",3,"40905,85,295",3,"30064,705,1499",3,"40905,85,295",3
95022,Zyvon,0,6,1.758859e-08,0.0,1.0,0.000000e+00,1.758859e-08,0.000000,0.000035,"40905,26982",2,"40905,26982",2,"40905,26982",2,"30064,20482",2,"40905,26982",2
95023,Zyyanna,6,0,1.758859e-08,1.0,0.0,1.758859e-08,0.000000e+00,0.000036,0.000000,"1168,22556,7697",3,"1168,22556,7697",3,"1168,22556,7697",3,"525,33175,4057",3,"1168,22556,7697",3
95024,Zyyon,0,6,1.758859e-08,0.0,1.0,0.000000e+00,1.758859e-08,0.000000,0.000035,"40905,19181",2,"40905,19181",2,"40905,19181",2,"30064,21743",2,"40905,19181",2


In [10]:
df_pivoted[df_pivoted["name"] == "Madison"]

gender,name,F,M,name_weight,F_prop,M_prop,F_weighted,M_weighted,F_weighted_norm,M_weighted_norm,gpt2-small,gpt2-small-size,gpt2-medium,gpt2-medium-size,gpt2-large,gpt2-large-size,facebook/opt-125m,facebook/opt-125m-size,EleutherAI/gpt-neo-125M,EleutherAI/gpt-neo-125M-size
56809,Madison,358035,7543,0.001072,0.979367,0.020633,0.00105,0.000022,2.118761,0.043817,14909,1,14909,1,14909,1,6370,1,14909,1


In [11]:
df_pivoted[ df_pivoted["M_weighted_norm"] < 0.01  ].sort_values("F_weighted_norm", ascending=False).head(20)

gender,name,F,M,name_weight,F_prop,M_prop,F_weighted,M_weighted,F_weighted_norm,M_weighted_norm,gpt2-small,gpt2-small-size,gpt2-medium,gpt2-medium-size,gpt2-large,gpt2-large-size,facebook/opt-125m,facebook/opt-125m-size,EleutherAI/gpt-neo-125M,EleutherAI/gpt-neo-125M-size
21538,Deborah,739654,1658,0.002173,0.997763,0.002237,0.002168,0.000005,4.377086,0.009631,36976,1,36976,1,36976,1,18878,1,36976,1
45737,Kathleen,710947,1693,0.002089,0.997624,0.002376,0.002084,0.000005,4.207205,0.009835,31275,1,31275,1,31275,1,15931,1,31275,1
27398,Emma,616404,1585,0.001812,0.997435,0.002565,0.001807,0.000005,3.647723,0.009207,18966,1,18966,1,18966,1,7957,1,18966,1
68522,Pamela,593850,1387,0.001745,0.997670,0.002330,0.001741,0.000004,3.514255,0.008057,43341,1,43341,1,43341,1,22998,1,43341,1
75429,Samantha,562160,1194,0.001651,0.997881,0.002119,0.001648,0.000004,3.326721,0.006936,34778,1,34778,1,34778,1,14155,1,34778,1
38159,Janet,555585,1518,0.001633,0.997275,0.002725,0.001629,0.000004,3.287812,0.008818,28111,1,28111,1,28111,1,11239,1,28111,1
14453,Carolyn,554145,1619,0.001629,0.997087,0.002913,0.001624,0.000005,3.279290,0.009405,43450,1,43450,1,43450,1,19949,1,43450,1
21552,Debra,550226,1146,0.001616,0.997922,0.002078,0.001613,0.000003,3.256099,0.006657,"8965,430",2,"8965,430",2,"8965,430",2,"10532,763",2,"8965,430",2
33547,Heather,524018,1482,0.001540,0.997180,0.002820,0.001536,0.000004,3.101006,0.008609,24253,1,24253,1,24253,1,10588,1,24253,1
23723,Diane,517797,1386,0.001522,0.997330,0.002670,0.001518,0.000004,3.064192,0.008051,26542,1,26542,1,26542,1,13649,1,26542,1


In [12]:
df_pivoted.to_csv(target_path, index=False)